In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [1]:
import numpy as np
import pandas as pd
import pickle, sys, os

from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Dense, Input, TimeDistributed, LSTM, Bidirectional, Embedding
from keras.layers import concatenate

from keras.models import Model

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

Using TensorFlow backend.


In [0]:
def word_to_boch(word):
    chars = list(word)
    res = np.zeros(shape=(MAX_WORD_LENGTH, ALPHABET_SIZE))
    for i, c in enumerate(chars):
        res[i][ALPHABET2INT[c]] = 1.0
    return res
  
def verbs_to_boch(inf, per1, per3):
    chars = [list(inf), list(per1), list(per3)]
    res = np.zeros(shape=(MAX_WORD_LENGTH, ALPHABET_SIZE * 3))
    for j in range(3):
      for i, c in enumerate(chars[j]):
        res[i][ALPHABET2INT[c] + j * ALPHABET_SIZE] = 1.0
    return res  

In [0]:
# PARAMETERS ================
ALPHABET_SIZE = 34
ALPHABET2INT = dict(zip(list('-абвгдеёжзийклмнопрстуфхцчшщъыьэюя'), range(34)))
INT2ALPHABET = dict(zip(range(34), list('-абвгдеёжзийклмнопрстуфхцчшщъыьэюя')))

MAX_WORD_LENGTH = 30

TEST_SPLIT = 0.2
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 32

In [4]:
!pip install graphviz

In [6]:
char_input = Input(shape=[MAX_WORD_LENGTH, ALPHABET_SIZE * 3], name="char_input")

features = Bidirectional(LSTM(MAX_WORD_LENGTH, return_sequences=True, name='lstm'), input_shape=(MAX_WORD_LENGTH, ALPHABET_SIZE * 3), name='bi')(char_input)

#modified_features = TimeDistributed(Dense(8, activation='tanh', name='tddense'))(features)

modified_features = Dense(8, activation='tanh', name='tddense')(features)

prediction = Dense(40, activation='softmax', name='final_dense')(modified_features)

model = Model(char_input, prediction)

model.compile('adam', validation_split=VALIDATION_SPLIT, loss='sparse_categorical_crossentropy', batch_size=BATCH_SIZE, metric='sparse_categorical_accuracy')

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
char_input (InputLayer)      (None, 30, 102)           0         
_________________________________________________________________
bi (Bidirectional)           (None, 30, 60)            31920     
_________________________________________________________________
tddense (Dense)              (None, 30, 8)             488       
_________________________________________________________________
final_dense (Dense)          (None, 30, 40)            360       
Total params: 32,768
Trainable params: 32,768
Non-trainable params: 0
_________________________________________________________________


In [11]:
!pip install pymorphy2

    100% |████████████████████████████████| 51kB 2.0MB/s 
    100% |████████████████████████████████| 7.1MB 828kB/s 
  Running setup.py bdist_wheel for docopt ... - done
  Stored in directory: /root/.cache/pip/wheels/9b/04/dd/7daf4150b6d9b12949298737de9431a324d4b797ffd63f526e
Successfully built docopt


In [0]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [0]:
def verb2tensor(inf):
  pm2 = morph.parse(inf)
  correct = None
  for var in pm2:
    if 'VERB' in var.tag or 'INFN' in var.tag:
      correct = var
      break
  per1 = correct.inflect({'1per'})[0] if correct.inflect({'1per'}) else ''
  per3 = correct.inflect({'3per'})[0] if correct.inflect({'3per'}) else ''
  
  
  res = verbs_to_boch(inf, per1, per3)
  
  return res

In [0]:
df = pd.read_csv('gdrive/My Drive/Colab Notebooks/short_verbs.csv')

In [57]:
df.head(10)

,word,accent,infltype1
0,жечь,2,8
1,выжечь,2,8г
2,возжечь,5,8
3,зажечь,4,8
4,дожечь,4,8
5,нажечь,4,8
6,обжечь,4,8
7,ожечь,3,8
8,пережечь,6,8
9,прижечь,5,8


In [0]:
X, y = df['word'].values, df['infltype1'].values

In [64]:
y

array(['8', '8г', '8', ..., '4', '4', '2'], dtype=object)

In [67]:
X[0]

'жечь'

In [0]:
X_emb = []
for verb in X:
  X_emb.append(verb2tensor(verb))
X_emb = np.array(X_emb)

In [76]:
X_emb.shape

(27166, 30, 102)

In [0]:
infl2int = dict()
int2infl = dict()

for i, elem in enumerate(set(y)):
  infl2int[elem] = i
  int2infl[i] = elem

In [85]:
y_ = np.array(list(map(lambda x: infl2int[x], y)))

y_emb = np.ndarray(shape=(y_.shape[0], 40))
print(y_emb.shape)
for i, elem in enumerate(y_):
  y_emb[i][elem] = 1.0

(27166, 40)


In [87]:
y_emb

array([[6.89553114e-316, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       ...,
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000]])

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_emb, y_emb, test_size=TEST_SPLIT)

In [93]:
X_train.shape, y_train.shape

((21732, 30, 102), (21732, 40))

In [102]:
model.fit(X_train, y_train)

ValueError: ignored